In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests

In [106]:

linkVec = []

for thisPage in [3]:#[2,3,4,5]:
    allLinkPage=requests.get("http://www.sirlinksalot.net/archives/thebachelor%d.html" % (thisPage))
    soup = BeautifulSoup(allLinkPage.text, "html.parser")
    rows = soup.find_all("a")
    for row in rows:
        rowString = str(row)
        checkASCII = [ord(letter) for letter in rowString]
        #print max(checkASCII),min(checkASCII)
        if max(checkASCII)<127 and min(checkASCII)>9:
        #    print max(checkASCII)<127 and min(checkASCII)>10
            thisLink = str(row.get("href"))
            if "sirlinksalot" not in thisLink: #remove internal links
                if "clickbank.net" not in thisLink: #remove ad links
                    if "pub43" not in thisLink: #remove ad links
                        if "bilbo" not in thisLink: #remove ad links
                            if "fastclick" not in thisLink: #remove ad links
                                if "casalemedia" not in thisLink: #remove ad links
                                    linkVec.append(thisLink)
                                


#cleaner = lambda r: [int(r[0].get_text()), r[1].get_text(), r[2].get_text(), r[2].find("a").get("href")]
#fields = ["ranking", "title", "band_singer", "url"]
#songs = [dict(zip(fields, cleaner(row.find_all("td")))) for row in rows]

In [157]:
#sort our vector to group similar website roots
linkVec = sorted(linkVec[0:20])

In [158]:
archivedLinkVec = []

count = 0;
length = len(linkVec)
for thisLink in linkVec:
    allLinkPage=requests.get("https://web.archive.org/web/*/" + thisLink)
    soup = BeautifulSoup(allLinkPage.text, "html.parser")
    if soup.find_all("div", attrs={"class": "date captures"}):
        row = soup.find_all("div", attrs={"class": "date captures"})[0]
        row = row.find("a").get("href")
        archivedLinkVec.append(str("https://web.archive.org" + row))
    count = count+1
    print count, " out of ", length, " done"

1  out of  20  done
2  out of  20  done
3  out of  20  done
4  out of  20  done
5  out of  20  done
6  out of  20  done
7  out of  20  done
8  out of  20  done
9  out of  20  done
10  out of  20  done
11  out of  20  done
12  out of  20  done
13  out of  20  done
14  out of  20  done
15  out of  20  done
16  out of  20  done
17  out of  20  done
18  out of  20  done
19  out of  20  done
20  out of  20  done


In [160]:
archivedLinkVec

['https://web.archive.org/web/20130103084536/http://blog.zap2it.com/frominsidethebox/2012/11/bachelor-finale-spoiler-who-does-sean-lowe-pick-and-are-they-engaged.html',
 'https://web.archive.org/web/20130110065255/http://blog.zap2it.com/frominsidethebox/2012/12/the-bachelor-premiere-sean-lowe-to-bend-the-rules-and-more-things-to-expect.html',
 'https://web.archive.org/web/20130128032851/http://blog.zap2it.com/frominsidethebox/2012/12/the-bachelor-spoilers-sean-lowe-tells-us-what-to-expect-in-season-17.html',
 'https://web.archive.org/web/20130131052218/http://insidetv.ew.com/2012/12/12/bachelor-sean-lowe/',
 'https://web.archive.org/web/20150906035505/http://www.dailymail.co.uk/tvshowbiz/article-2243684/The-Bachelor-2013-Models-galore-cruise-ship-entertainer-poker-dealer-Meet-ladies-compete-win-hunky-Sean-Lowes-heart.html?ito=feeds-newsxml',
 'https://web.archive.org/web/20150906030546/http://www.digitalspy.com/celebrity/s77/the-bachelor/news/a442707/bachelor-molly-mesnick-feeling-real

In [ ]:
#we can assume that each article from the Examiner website will have similar HTML formatting.